In [ ]:
!pip install datasets==2.16.1

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      Successfully uninstalled fsspec-2024.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x

In [ ]:
from transformers import pipeline

model_checkpoint = "Helsinki-NLP/opus-mt-en-fr"
translator = pipeline("translation", model=model_checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/301M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
translator("Default to expanded threads"),\
translator(
    "Unable to import %1 using the OFX importer plugin. This file is not the correct format."
)

([{'translation_text': 'Par défaut pour les threads élargis'}],
 [{'translation_text': "Impossible d'importer %1 en utilisant le plugin d'importateur OFX. Ce fichier n'est pas le bon format."}])

In [ ]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
data = load_dataset("kde4", lang1="en", lang2="fr")
data = data['train'].train_test_split(0.1)


checkpoint = 'Helsinki-NLP/opus-mt-en-fr'

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

data

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for kde4 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/kde4
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/210173 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 189155
    })
    test: Dataset({
        features: ['id', 'translation'],
        num_rows: 21018
    })
})

In [ ]:

def split_en_fr(examples):
  input = [sample['en'] for sample in examples['translation']]
  target = [sample['fr'] for sample in examples['translation']]
  return {'input':input,'target':target}


data = data.map(split_en_fr,batched=True,remove_columns=data['train'].column_names)
data

Map:   0%|          | 0/189155 [00:00<?, ? examples/s]

Map:   0%|          | 0/21018 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input', 'target'],
        num_rows: 189155
    })
    test: Dataset({
        features: ['input', 'target'],
        num_rows: 21018
    })
})

In [ ]:

def process_and_tokenize(examples):
  input_data = tokenizer(examples['input'],truncation=True,max_length=128)

  with tokenizer.as_target_tokenizer():
    target_data = tokenizer(examples['target'],truncation=True,max_length=128)

  input_data['labels'] = target_data['input_ids']
  return input_data

data = data.map(process_and_tokenize,batched=True,remove_columns=['input','target'])

Map:   0%|          | 0/189155 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4016: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/21018 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer,model=model,return_tensors='pt')

In [ ]:
from huggingface_hub import notebook_login


notebook_login()

In [ ]:
!pip install sacreblu
!pip install evaluate

ERROR: Could not find a version that satisfies the requirement sacreblu (from versions: none)
ERROR: No matching distribution found for sacreblu


In [ ]:
import evaluate
import numpy as np



metric = evaluate.load('sacrebleu')

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

def compute_metrics(eval_preds):
  preds,labels = eval_preds
  if isinstance(preds,tuple):
    preds = preds[0]
  # preds = preds.argmax(-1)

  preds = tokenizer.batch_decode(preds,skip_special_tokens=True)

  labels = np.where(labels!=-100,labels,tokenizer.pad_token_id)
  labels = tokenizer.batch_decode(labels,skip_special_tokens=True)
  result = metric.compute(predictions=preds,references=labels)
  return {'blue':result['score']}

args = Seq2SeqTrainingArguments(
   'En-Fr-Translation-Computer-Science-Domain',
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=data["train"],
    eval_dataset=data["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)



/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer.train()

Step,Training Loss
500,0.960400
1000,0.974200
1500,0.964600
2000,0.945000
2500,0.946700
3000,0.925500
3500,0.933000
4000,0.916500
4500,0.923900
5000,0.903100


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[59513]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[59513]], 'forced_eos_token_id': 0}


Step,Training Loss
500,0.960400
1000,0.974200
1500,0.964600
2000,0.945000
2500,0.946700
3000,0.925500
3500,0.933000
4000,0.916500
4500,0.923900
5000,0.903100


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[59513]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[59513]], 'forced_eos_token_id': 0}


TrainOutput(global_step=17736, training_loss=0.8326902755166047, metrics={'train_runtime': 1213.0312, 'train_samples_per_second': 467.807, 'train_steps_per_second': 14.621, 'total_flos': 1.1303936534052864e+16, 'train_loss': 0.8326902755166047, 'epoch': 3.0})

In [ ]:
max_length = 128
trainer.evaluate(max_length = max_length)

{'eval_loss': 0.8427127599716187,
 'eval_blue': 54.050212016402725,
 'eval_runtime': 1021.3904,
 'eval_samples_per_second': 20.578,
 'eval_steps_per_second': 0.322,
 'epoch': 3.0}

In [ ]:
trainer.push_to_hub()

In [ ]:
hamza_translator = pipeline('translation','HamzaNaser/En-Fr-Translation-Computer-Science-Domain')

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/299M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/288 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/816 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.46M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
hamza_translator("Default to expanded threads"),\
hamza_translator(
    "Unable to import %1 using the OFX importer plugin. This file is not the correct format."
)

([{'translation_text': 'Par défaut, développer les fils de discussion'}],
 [{'translation_text': "Impossible d'importer %1 en utilisant le module externe d'importation OFX. Ce fichier n'est pas le bon format."}])

([{'translation_text': 'Par défaut, développer les fils de discussion'}],
 [{'translation_text': "Impossible d'importer %1 en utilisant le module externe d'importation OFX. Ce fichier n'est pas le bon format."}])

In [ ]:
5+5

10